In [1]:
import pandas as pd
import numpy as np
import matplotlib

# import the celeb csv file.
comments_df = pd.read_csv('../../datasets/raw/youtube_comments.csv')

# import celeb csv file.
celeb_df = pd.read_csv("../../datasets/attendees.csv")

# check is imported.
comments_df.head(5)
celeb_df.head(5)

,Name,Year,Gender
0,Billie Eilish,2021,Unknown
1,A$AP Rocky,2021,Male
2,Rihanna,2021,Female
3,Jennifer Lopez,2021,Female
4,Lil Nas X,2021,Male


In [2]:
import json
import pandas as pd
import re
import networkx as nx
from itertools import combinations
import os

print(f"Attendees CSV columns: {list(celeb_df.columns)}")
print(f"Comments CSV columns: {list(comments_df.columns)}")

# --- Step 2: Text preprocessing function (for celebrity names only) ---
def preprocess_text(text):
    """Preprocess celebrity names for better matching with already-preprocessed comments"""
    if pd.isna(text) or text == "":
        return ""
    
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove extra whitespace and normalize
    text = re.sub(r'\s+', ' ', text)
    
    # Remove common punctuation that might interfere with name matching
    text = re.sub(r'[^\w\s\'-]', ' ', text)
    
    # Handle common variations
    text = re.sub(r'\bkim\s*k\b', 'kim kardashian', text)
    text = re.sub(r'\bkardashian\b', 'kim kardashian', text)  
    text = re.sub(r'\bzendaya\b', 'zendaya', text)
    text = re.sub(r'\btaylor\s*swift\b', 'taylor swift', text)
    text = re.sub(r'\bt\s*swift\b', 'taylor swift', text)
    text = re.sub(r'\bbeyonce\b', 'beyonce', text)
    text = re.sub(r'\bbey\b', 'beyonce', text)
    text = re.sub(r'\bjlo\b', 'jennifer lopez', text)
    text = re.sub(r'\bj\s*lo\b', 'jennifer lopez', text)
    text = re.sub(r'\bariana\b', 'ariana grande', text)
    text = re.sub(r'\bari\b', 'ariana grande', text)
    text = re.sub(r'\bselena\b', 'selena gomez', text)
    text = re.sub(r'\briri\b', 'rihanna', text)
    text = re.sub(r'\bgaga\b', 'lady gaga', text)
    text = re.sub(r'\blady\s*gaga\b', 'lady gaga', text)
    text = re.sub(r'\bdua\b', 'dua lipa', text)
    text = re.sub(r'\bbillie\b', 'billie eilish', text)
    text = re.sub(r'\bthe\s*weeknd\b', 'the weeknd', text)
    text = re.sub(r'\bweeknd\b', 'the weeknd', text)
    
    return text.strip()

# Create mapping of celebrity -> years they attended
# Only preprocess celebrity names, comments are already preprocessed
celeb_year_mapping = {}
for index, row in celeb_df.iterrows():
    name = row.get('Name')
    year = row.get('Year')  # Attendees CSV uses 'Year' column (capital Y)
    
    if pd.notna(name) and pd.notna(year):
        processed_name = preprocess_text(name)  # Preprocess celebrity name to match preprocessed comments
        if processed_name:
            if processed_name not in celeb_year_mapping:
                celeb_year_mapping[processed_name] = set()
            celeb_year_mapping[processed_name].add(year)

print(f"\nLoaded celebrity attendance data for {len(celeb_year_mapping)} celebrities:")
for celeb, years in list(celeb_year_mapping.items())[:10]:  # Show first 10 as example
    print(f"  {celeb}: {sorted(years)}")
if len(celeb_year_mapping) > 10:
    print(f"  ... and {len(celeb_year_mapping) - 10} more")

# --- Step 3: Setup celebrity patterns (full names only) ---
raw_celeb_names = celeb_df['Name'].dropna().unique()

# Create mapping system for full names only
celeb_name_mapping = {}
all_patterns = []

for original_name in raw_celeb_names:
    processed_name = preprocess_text(original_name)
    if not processed_name:
        continue
    
    # Store the canonical name (full name only)
    canonical_name = processed_name
    
    # Add full name pattern only
    celeb_name_mapping[processed_name] = canonical_name
    all_patterns.append((processed_name, canonical_name, len(processed_name)))

# Sort patterns by length (longest first) for better matching
all_patterns.sort(key=lambda x: x[2], reverse=True)
pattern_strings = [re.escape(pattern[0]) for pattern in all_patterns]

celeb_regex = re.compile(r'\b(?:' + '|'.join(pattern_strings) + r')\b', re.IGNORECASE)

print(f"Loaded {len(raw_celeb_names)} celebrity names")
print(f"Created {len(pattern_strings)} search patterns (full names only)")
print(f"Loaded {len(comments_df)} comments")

# --- Step 4: Create output directory ---
output_dir = "comention_graphs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Step 6: Function to extract celebrity mentions with year filtering ---
def extract_mentions(text, comment_year):
    """Extract celebrity mentions from text (text is already preprocessed)"""
    if pd.isna(text) or text == "":
        return []
    
    # Text is already preprocessed, use as-is
    processed_text = str(text).lower()
    
    # Find all matches with their positions
    matches = []
    for match in celeb_regex.finditer(processed_text):
        match_text = match.group().lower()
        if match_text in celeb_name_mapping:
            celebrity_name = celeb_name_mapping[match_text]
            
            # Only include if celebrity attended in this comment's year
            if celebrity_name in celeb_year_mapping and comment_year in celeb_year_mapping[celebrity_name]:
                matches.append((match.start(), match.end(), match_text, celebrity_name))
    
    # Remove overlapping matches (prefer longer matches)
    matches.sort(key=lambda x: x[0])
    
    non_overlapping = []
    for i, (start, end, match_text, canonical) in enumerate(matches):
        # Check if this match overlaps with any previously accepted match
        overlaps = False
        for prev_start, prev_end, _, _ in non_overlapping:
            if not (end <= prev_start or start >= prev_end):  # They overlap
                overlaps = True
                break
        
        if not overlaps:
            non_overlapping.append((start, end, match_text, canonical))
    
    # Extract unique celebrity names (remove duplicates where same person mentioned multiple times)
    celebrity_mentions = set()
    for _, _, _, canonical in non_overlapping:
        celebrity_mentions.add(canonical)
    
    return list(celebrity_mentions)

# --- Step 5: Function to create co-mention graph ---
def create_comention_graph(df, year_filter=None):
    """Create co-mention graph for celebrities mentioned together"""
    G = nx.Graph()
    
    # Filter by year if specified
    if year_filter:
        df = df[df['met_gala_year'] == year_filter]
        graph_name = f"Met Gala {year_filter}"
    else:
        graph_name = "All Years"
    
    print(f"\n=== Processing {graph_name} ===")
    print(f"Processing {len(df)} comments...")
    
    mention_count = 0
    total_mentions = 0
    
    # Process each comment
    for index, row in df.iterrows():
        # Extract text and year from the comment
        comment_text = str(row.get('text', ''))
        comment_year = row.get('met_gala_year')  # Extract the comment year!
        
        if pd.isna(comment_year):
            continue
        
        # Find mentioned celebrities (filtered by year they attended)
        mentioned = extract_mentions(comment_text, comment_year)
        total_mentions += len(mentioned)
        
        if len(mentioned) > 1:
            mention_count += 1
            # Create edges for all pairs of mentioned celebrities
            for celeb1, celeb2 in combinations(sorted(mentioned), 2):
                if G.has_edge(celeb1, celeb2):
                    G[celeb1][celeb2]['weight'] += 1
                else:
                    G.add_edge(celeb1, celeb2, weight=1)
    
    return G, mention_count, total_mentions, graph_name

# --- Step 6: Function to analyze graph ---
def analyze_comention_graph(G, mention_count, total_mentions, graph_name, df_size):
    """Analyze and print co-mention graph statistics"""
    print(f"\n=== {graph_name} Co-mention Graph Summary ===")
    print(f"Total comments processed: {df_size}")
    print(f"Total celebrity mentions: {total_mentions}")
    print(f"Comments with multiple celebrity mentions: {mention_count}")
    print(f"Nodes (Celebrities): {G.number_of_nodes()}")
    print(f"Edges (Co-mentions): {G.number_of_edges()}")
    
    if G.number_of_edges() == 0:
        print("No co-mentions found for this period.")
        return
    
    # Top co-mentions
    print(f"\n--- Top 10 Celebrity Co-mentions ({graph_name}) ---")
    edges_with_weights = [(u, v, data['weight']) for u, v, data in G.edges(data=True)]
    edges_with_weights.sort(key=lambda x: x[2], reverse=True)
    
    for i, (celeb1, celeb2, weight) in enumerate(edges_with_weights[:10]):
        print(f"{i+1}. {celeb1.title()} & {celeb2.title()}: {weight} co-mentions")
    
    # Most connected celebrities
    print(f"\n--- Most Connected Celebrities ({graph_name}) ---")
    if G.number_of_nodes() > 0:
        degrees = dict(G.degree())
        sorted_degrees = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
        
        for i, (celeb, degree) in enumerate(sorted_degrees[:10]):
            print(f"{i+1}. {celeb.title()}: {degree} connections")
    
    # Additional statistics
    if G.number_of_nodes() > 0:
        print(f"\n--- Network Statistics ({graph_name}) ---")
        print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")
        
        # Most frequently co-mentioned pairs
        total_weight = sum([data['weight'] for u, v, data in G.edges(data=True)])
        print(f"Total co-mention instances: {total_weight}")
        
        if G.number_of_edges() > 0:
            avg_weight = total_weight / G.number_of_edges()
            print(f"Average co-mentions per pair: {avg_weight:.2f}")

# --- Step 7: Function to save graph ---
def save_comention_graph(G, graph_name, year_filter=None):
    """Save co-mention graph as GraphML file"""
    if year_filter:
        filename = f"celeb_comentions_{year_filter}.graphml"
    else:
        filename = "celeb_comentions_all_years.graphml"
    
    filepath = os.path.join(output_dir, filename)
    nx.write_graphml(G, filepath)
    print(f"Saved {graph_name} graph: {filepath}")

# --- Step 8: Create and analyze all-years graph ---
G_all, mentions_all, total_mentions_all, name_all = create_comention_graph(comments_df)
analyze_comention_graph(G_all, mentions_all, total_mentions_all, name_all, len(comments_df))
save_comention_graph(G_all, name_all)

# --- Step 9: Create and analyze individual year graphs ---
unique_years = sorted(comments_df['met_gala_year'].dropna().unique())
print(f"\n=== Found {len(unique_years)} unique years: {unique_years} ===")

year_graphs = {}
for year in unique_years:
    year_df = comments_df[comments_df['met_gala_year'] == year]
    G_year, mentions_year, total_mentions_year, name_year = create_comention_graph(comments_df, year)
    
    if G_year.number_of_nodes() > 0:  # Only analyze if graph has data
        analyze_comention_graph(G_year, mentions_year, total_mentions_year, name_year, len(year_df))
        save_comention_graph(G_year, name_year, year)
        year_graphs[year] = G_year
    else:
        print(f"No co-mentions found for {year}")

# --- Step 10: Cross-year comparison ---
print(f"\n=== Cross-Year Co-mention Comparison ===")
for year in unique_years:
    if year in year_graphs:
        G = year_graphs[year]
        total_weight = sum([data['weight'] for u, v, data in G.edges(data=True)])
        print(f"{year}: {G.number_of_nodes()} celebrities, {G.number_of_edges()} pairs, {total_weight} total co-mentions")

# --- Step 11: Most mentioned celebrities across all data (with year filtering) ---
print(f"\n=== Most Mentioned Celebrities (All Years, Year-Filtered) ===")
all_celebrity_mentions = []
for index, row in comments_df.iterrows():
    comment_text = str(row.get('text', ''))
    comment_year = row.get('met_gala_year')
    
    if pd.notna(comment_year):
        mentioned = extract_mentions(comment_text, comment_year)
        all_celebrity_mentions.extend(mentioned)

if all_celebrity_mentions:
    from collections import Counter
    mention_counts = Counter(all_celebrity_mentions)
    
    for i, (celeb, count) in enumerate(mention_counts.most_common(15)):
        # Show which years this celebrity attended
        years_attended = sorted(celeb_year_mapping.get(celeb, []))
        print(f"{i+1}. {celeb.title()}: mentioned {count} times (attended: {years_attended})")
else:
    print("No valid celebrity mentions found after year filtering")

# --- Step 12: Year-specific mention analysis ---
print(f"\n=== Celebrity Mentions by Year ===")
for year in unique_years:
    year_comments = comments_df[comments_df['met_gala_year'] == year]
    year_mentions = []
    
    for index, row in year_comments.iterrows():
        comment_text = str(row.get('text', ''))
        comment_year = row.get('met_gala_year')
        if pd.notna(comment_year):
            mentioned = extract_mentions(comment_text, comment_year)
            year_mentions.extend(mentioned)
    
    if year_mentions:
        year_counts = Counter(year_mentions)
        print(f"\n{year} - Top mentioned celebrities:")
        for i, (celeb, count) in enumerate(year_counts.most_common(5)):
            print(f"  {i+1}. {celeb.title()}: {count} mentions")
    else:
        print(f"\n{year} - No valid celebrity mentions found")

print(f"\n=== Analysis Complete! ===")
print(f"Year filtering applied: Celebrities only counted in years they attended")
print(f"GraphML files saved in '{output_dir}' directory:")
print(f"- All-years graph: celeb_comentions_all_years.graphml")
for year in unique_years:
    if year in year_graphs:
        print(f"- {year} graph: celeb_comentions_{year}.graphml")

Attendees CSV columns: ['Name', 'Year', 'Gender']
Comments CSV columns: ['video_id', 'text', 'author', 'likes', 'date', 'time', 'year', 'met_gala_year']

Loaded celebrity attendance data for 418 celebrities:
  billie eilish eilish: [2021, 2022]
  a ap rocky: [2021, 2023, 2025]
  rihanna: [2021, 2023, 2025]
  jennifer lopez: [2021, 2023]
  lil nas x: [2021, 2023, 2024]
  lorde: [2021, 2025]
  timothée chalamet: [2021]
  iman: [2021]
  kaia gerber: [2021, 2022]
  kendall jenner: [2021, 2022, 2023, 2024, 2025]
  ... and 408 more
Loaded 418 celebrity names
Created 418 search patterns (full names only)
Loaded 72076 comments

=== Processing All Years ===
Processing 72076 comments...

=== All Years Co-mention Graph Summary ===
Total comments processed: 72076
Total celebrity mentions: 8591
Comments with multiple celebrity mentions: 653
Nodes (Celebrities): 232
Edges (Co-mentions): 2453

--- Top 10 Celebrity Co-mentions (All Years) ---
1. Zendaya & Diana Ross: 28 co-mentions
2. Rihanna & Lil Na

In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import os

# --- Step 1: Load YouTube comments data ---
print(f"Loaded {len(comments_df)} comments")

# --- Step 2: Create output directory for graphs ---
output_dir = "reply_graphs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Step 3: Function to create directed graph ---
def create_directed_graph(df, year_filter=None):
    """Create directed graph where users point to videos they commented on"""
    G = nx.DiGraph()  # Directed graph
    
    # Filter by year if specified
    if year_filter:
        df = df[df['met_gala_year'] == year_filter]
        graph_name = f"Met Gala {year_filter}"
    else:
        graph_name = "All Years"
    
    # Track node types
    video_nodes = set()
    user_nodes = set()
    video_info = {}
    
    print(f"\n=== Processing {graph_name} ===")
    print(f"Processing {len(df)} comments...")
    
    for index, row in df.iterrows():
        video_id = str(row.get('video_id', ''))
        author = str(row.get('author', ''))
        met_gala_year = str(row.get('met_gala_year', ''))
        
        # Skip if essential data is missing
        if pd.isna(video_id) or pd.isna(author) or video_id == '' or author == '':
            continue
        
        # Create video node (just the video_id as requested)
        video_node = f"{video_id}_{met_gala_year}"
        video_info[video_node] = (video_id, met_gala_year)
        
        # Add nodes to respective sets
        video_nodes.add(video_node)
        user_nodes.add(author)
        
        # Add directed edge: user -> video (user comments on video)
        if G.has_edge(author, video_node):
            G[author][video_node]['weight'] += 1
        else:
            G.add_edge(author, video_node, weight=1)
    
    return G, video_nodes, user_nodes, video_info, graph_name

# --- Step 4: Function to analyze graph ---
def analyze_graph(G, video_nodes, user_nodes, video_info, graph_name):
    """Analyze and print graph statistics"""
    print(f"\n=== {graph_name} Graph Summary ===")
    print(f"Unique videos: {len(video_nodes)}")
    print(f"Unique users: {len(user_nodes)}")
    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Total edges (user → video): {G.number_of_edges()}")
    
    if G.number_of_nodes() == 0:
        print("No data to analyze for this period.")
        return
    
    # Most active users (highest out-degree - commenting on most videos)
    print(f"\n--- Most Active Users ({graph_name}) ---")
    user_out_degrees = {}
    for node in G.nodes():
        if node in user_nodes:
            user_out_degrees[node] = G.out_degree(node)
    
    if user_out_degrees:
        sorted_users = sorted(user_out_degrees.items(), key=lambda x: x[1], reverse=True)
        for i, (user, degree) in enumerate(sorted_users[:10]):
            print(f"{i+1}. {user}: commented on {degree} videos")
    
    # Most popular videos (highest in-degree - most users commenting)
    print(f"\n--- Most Popular Videos ({graph_name}) ---")
    video_in_degrees = {}
    for node in G.nodes():
        if node in video_nodes:
            video_in_degrees[node] = G.in_degree(node)
    
    if video_in_degrees:
        sorted_videos = sorted(video_in_degrees.items(), key=lambda x: x[1], reverse=True)
        for i, (video_node, degree) in enumerate(sorted_videos[:10]):
            video_id, year = video_info[video_node]
            print(f"{i+1}. Video {video_id} ({year}): {degree} unique commenters")
    
    # Comment volume analysis (edge weights)
    print(f"\n--- Highest Comment Volumes ({graph_name}) ---")
    edges_with_weights = []
    for user, video, data in G.edges(data=True):
        if user in user_nodes:  # user -> video edge
            video_id, year = video_info[video]
            edges_with_weights.append((user, video_id, year, data['weight']))
    
    if edges_with_weights:
        edges_with_weights.sort(key=lambda x: x[3], reverse=True)
        for i, (user, video_id, year, weight) in enumerate(edges_with_weights[:10]):
            print(f"{i+1}. {user} → Video {video_id} ({year}): {weight} comments")

# --- Step 5: Function to save graph ---
def save_graph(G, video_nodes, user_nodes, video_info, graph_name, year_filter=None):
    """Save graph as GraphML file"""
    if year_filter:
        filename_base = f"reply_graph_{year_filter}"
    else:
        filename_base = "reply_graph_all_years"
    
    # Save as GraphML
    graphml_path = os.path.join(output_dir, f"{filename_base}.graphml")
    nx.write_graphml(G, graphml_path)
    
    print(f"Saved {graph_name} graph: {graphml_path}")

# --- Step 6: Create and analyze all-years graph ---
G_all, video_nodes_all, user_nodes_all, video_info_all, name_all = create_directed_graph(comments_df)
analyze_graph(G_all, video_nodes_all, user_nodes_all, video_info_all, name_all)
save_graph(G_all, video_nodes_all, user_nodes_all, video_info_all, name_all)

# --- Step 7: Create and analyze individual year graphs ---
unique_years = sorted(comments_df['met_gala_year'].dropna().unique())
print(f"\n=== Found {len(unique_years)} unique years: {unique_years} ===")

year_graphs = {}
for year in unique_years:
    G_year, video_nodes_year, user_nodes_year, video_info_year, name_year = create_directed_graph(comments_df, year)
    
    if G_year.number_of_nodes() > 0:  # Only analyze if graph has data
        analyze_graph(G_year, video_nodes_year, user_nodes_year, video_info_year, name_year)
        save_graph(G_year, video_nodes_year, user_nodes_year, video_info_year, name_year, year)
        year_graphs[year] = G_year
    else:
        print(f"No data found for {year}")

# --- Step 8: Cross-year comparison ---
print(f"\n=== Cross-Year Comparison ===")
for year in unique_years:
    if year in year_graphs:
        G = year_graphs[year]
        print(f"{year}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# --- Step 9: Users active across multiple years ---
print(f"\n=== Users Active Across Multiple Years ===")
user_years = comments_df.groupby('author')['met_gala_year'].nunique().sort_values(ascending=False)
multi_year_users = user_years[user_years > 1]

if len(multi_year_users) > 0:
    for user, year_count in multi_year_users.head(10).items():
        years = sorted(comments_df[comments_df['author'] == user]['met_gala_year'].unique())
        total_comments = len(comments_df[comments_df['author'] == user])
        print(f"- {user}: active in {year_count} years ({', '.join(map(str, years))}) - {total_comments} total comments")
else:
    print("No users found commenting across multiple years")

print(f"\n=== Analysis Complete! ===")
print(f"GraphML files saved in '{output_dir}' directory:")
print(f"- All-years graph: reply_graph_all_years.graphml")
for year in unique_years:
    if year in year_graphs:
        print(f"- {year} graph: reply_graph_{year}.graphml")

Loaded 72076 comments

=== Processing All Years ===
Processing 72076 comments...

=== All Years Graph Summary ===
Unique videos: 22
Unique users: 52105
Total nodes: 52127
Total edges (user → video): 56224

--- Most Active Users (All Years) ---
1. nan: commented on 12 videos
2. @levjerraz6983: commented on 9 videos
3. @pollycipher: commented on 7 videos
4. @krishniarumugam: commented on 7 videos
5. @babybluecheeks: commented on 7 videos
6. @johnpearson1258: commented on 6 videos
7. @hautelemode: commented on 6 videos
8. @duchessedeberne3909: commented on 6 videos
9. @CARATMom: commented on 6 videos
10. @charlottemccaig578: commented on 6 videos

--- Most Popular Videos (All Years) ---
1. Video NW2oiPiqByk (2025): 8300 unique commenters
2. Video ZMrgtotgThk (2021): 7189 unique commenters
3. Video PbRZcvVnF0w (2022): 5294 unique commenters
4. Video P71sr0kZY7o (2024): 4566 unique commenters
5. Video AyFzKATCiv0 (2025): 4070 unique commenters
6. Video jlR-T42I18E (2024): 3086 unique commen